# C-Python App

In [1]:
from app import app_main

await app_main()

{'version': '2024-08-31T07:59:54.035268', 'secrets': {'api_keys': ['klasdfj9y7984q2352hjkfoahjfk'], 'wifi': [{'ssid': 'R', 'password': 'jdqwn4x84iouc'}, {'ssid': 'R-5G', 'password': 'jdqwn4x84iouc'}, {'ssid': 'TPA-R', 'password': 'fbfYZBUMRcvZxd498kK4'}, {'ssid': 'TPA-IoT', 'password': 'grusig-verruckt-17'}, {'ssid': 'TPA', 'password': 'TurbenThal'}]}, 'leaves': {'new_leaf': {'mac_addr': '19:05:e2:48:e5:1d'}, 'ble_leaf': {'mac_addr': 'a6:e5:b4:11:f2:bb', 'plugins': {'plugins.ble_scanner': {'duration_ms': 500, 'active': True, 'devices': {'a4:c1:38:ff:64:24': {'id': 'fridge_top'}, 'a4:c1:38:18:a6:fe': {'id': 'fridge_bot'}, 'a4:c1:38:c8:cf:3d': {'id': 'freezer'}, 'a4:c1:38:97:7c:88': {'id': 'utility_bay'}, 'f4:d8:cb:f5:70:75': {'id': 'soc', 'key': '8aa525733cee8edcacd31eade2a6efa7'}, 'fb:f9:ca:9e:30:bd': {'alias': 'mppt', 'key': '919c8da4bb4e59d801e22efd13ba78ee'}, 'a4:c1:38:7f:05:15': {'id': 'bv_freezer'}, 'a4:c1:38:26:8c:84': {'id': 'bv_fridge'}, 'f5:78:d2:09:01:de': {'id': 'bv_soc', 'k

CancelledError: 